In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import os, datetime, torch
from typing import *
from recstudio.utils import *
import torch.nn as nn

In [21]:
import recstudio.eval as eval
model, dataset = 'Test10', 'amazon-beauty'
model_class, model_conf = get_model(model)
model = model_class(model_conf)
dataset_class = model_class._get_dataset_class()

data_conf = {}

data_conf.update(model_conf['data'])    # update model-specified config
datasets = dataset_class(name=dataset, config=data_conf).build(**model_conf['data'])
train_data = datasets[0]


import logging
logger = get_logger('./test.log')
logger.setLevel(logging.ERROR)
model.logger = logging.getLogger('recstudio')

model._init_model(train_data)
model._init_parameter()
model.run_mode = 'light'
model.val_check = True
val_metrics = model.config['eval']['val_metrics']
model.val_metric = next(iter(val_metrics)) if isinstance(val_metrics, list) else val_metrics
cutoff = model.config['eval']['cutoff']
cutoffs = cutoff if isinstance(cutoff, list) else [cutoff]
if len(eval.get_rank_metrics(model.val_metric)) > 0:
    model.val_metric += '@' + str(cutoffs[0])
model.callback = model._get_callback(train_data.name)
model._accelerate()

In [22]:
save_dir = model.config['eval']['save_path']
model.save_dir = save_dir
ckpt = './saved/Test10/amazon-beauty/2023-05-16-15-36-54-098206.ckpt'
model.ckpt_path = './Test10/amazon-beauty/2023-05-16-15-36-54-098206.ckpt'
model.load_checkpoint(ckpt)
model.trainloaders = model._get_train_loaders(train_data)
model.optimizers = model._get_optimizers()

In [23]:
model.config['model']['cl_weight'] = 1


In [ ]:
for g in model.optimizers[0]['optimizer'].param_groups:
    g['lr'] = 0.001

In [25]:
model.finetune = True
model.query_encoder.finetune = True
for _ in range(100):
    out = model.evaluate(datasets[-1])
    print(out['ndcg@20'])
    print(out['recall@20'])

test
tensor(0.0489)
tensor(0.1016)
test
tensor(0.0492)
tensor(0.1019)
test
tensor(0.0487)
tensor(0.1015)
test
tensor(0.0490)
tensor(0.1020)
test
tensor(0.0491)
tensor(0.1016)
test
tensor(0.0493)
tensor(0.1020)
test
tensor(0.0497)
tensor(0.1022)
test
tensor(0.0494)
tensor(0.1023)
test
tensor(0.0481)
tensor(0.1010)
test
tensor(0.0487)
tensor(0.1010)
